In [1]:
text = open("tests/Kannada2023.txt", "r", encoding="utf-8").read()
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience

In [2]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 5024 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  # print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

In [3]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 169342
ids length: 13213
compression ratio: 12.82X


### decoding

Given a sequence of integers in the range [0, vocab_size], what is the text?


In [4]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([128]))

�


### encoding

The other way around: Given a string, what are the tokens?


In [5]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(encode(""))

[]


In [7]:
print(decode(encode("ನಮಸ್ಕಾರವನ್ನು ಸಾಮಾನ್ಯವಾಗಿ ಸ್ವಲ್ಪ ಬಗ್ಗಿ ಮತ್ತು ಕೈಗಳನ್ನು ಒಂದಕ್ಕೊಂದು ಒತ್ತಿ, ಅಂಗೈಗಳು ಸ್ಪರ್ಶಿಸಿ ಮತ್ತು ಬೆರಳುಗಳು ಮೇಲ್ಮುಖವಾಗಿ ಇದ್ದು, ಹೆಬ್ಬೆರಳುಗಳನ್ನು ಎದೆಗೆ ಹತ್ತಿರವಿರಿಸಿ ಹೇಳಲಾಗುತ್ತದೆ. ಈ ಸನ್ನೆಯನ್ನು ಅಂಜಲಿ ಮುದ್ರೆ ಅಥವಾ ಪ್ರಣಾಮಾಸನ ಎಂದು ಕರೆಯಲಾಗುತ್ತದೆ.")))

ನಮಸ್ಕಾರವನ್ನು ಸಾಮಾನ್ಯವಾಗಿ ಸ್ವಲ್ಪ ಬಗ್ಗಿ ಮತ್ತು ಕೈಗಳನ್ನು ಒಂದಕ್ಕೊಂದು ಒತ್ತಿ, ಅಂಗೈಗಳು ಸ್ಪರ್ಶಿಸಿ ಮತ್ತು ಬೆರಳುಗಳು ಮೇಲ್ಮುಖವಾಗಿ ಇದ್ದು, ಹೆಬ್ಬೆರಳುಗಳನ್ನು ಎದೆಗೆ ಹತ್ತಿರವಿರಿಸಿ ಹೇಳಲಾಗುತ್ತದೆ. ಈ ಸನ್ನೆಯನ್ನು ಅಂಜಲಿ ಮುದ್ರೆ ಅಥವಾ ಪ್ರಣಾಮಾಸನ ಎಂದು ಕರೆಯಲಾಗುತ್ತದೆ.


In [8]:
text2 = decode(encode(text))
print(text2 == text)

True


In [3]:
# import gradio as gr
from minbpe.regex import RegexTokenizer

def inference(text):
    tokenizer = RegexTokenizer()
    tokenizer.load("./models/regex.model")
    tokens = tokenizer.encode(text)
    count = len(tokens)
    return count, tokens



In [4]:
text = "ಈ ನಮಸ್ಕಾರ ಮಾಡುವುದರಿಂದ ಅನೇಕ ವೈಜ್ಞಾನಿಕ, ಆಧ್ಯಾತ್ಮಿಕ ಪ್ರಯೋಜನಗಳಿವೆ."
inference(text)

(29,
 [1137,
  335,
  317,
  279,
  1005,
  299,
  4011,
  264,
  741,
  419,
  1117,
  3476,
  1003,
  970,
  44,
  637,
  437,
  278,
  635,
  631,
  357,
  330,
  296,
  1069,
  698,
  845,
  262,
  534,
  46])